In [4]:
import os

In [2]:
%pwd

'/home/vaibhav/Chest_Cancer_Classification_Using_MLFlow-DVC/research'

In [5]:
os.chdir("../")

In [5]:
%pwd

'/home/vaibhav/Chest_Cancer_Classification_Using_MLFlow-DVC'

In [7]:
#update entity module

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_url : str 
    local_data_file : Path 
    unzip_dir: Path 

In [8]:
# Update the configuration manager in src config
from Chest_Cancer_Classification.constants import *
from Chest_Cancer_Classification.utils.common import read_yaml,create_directories

In [23]:



class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])       

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        root_dir = config.root_dir,
        source_url = config.source_url,
        local_data_file = config.local_data_file,
        unzip_dir=config.unzip_dir)

        return data_ingestion_config

In [24]:
import os
import zipfile
import gdown
from Chest_Cancer_Classification import logger
from Chest_Cancer_Classification.utils.common import get_size
from Chest_Cancer_Classification.entity.config_entity import DataIngestionConfig

In [26]:
# update component
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [27]:
# update pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 



[2024-02-26 10:45:20,919: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-26 10:45:20,921: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-26 10:45:20,922: INFO: common: created directory at: artifacts]
[2024-02-26 10:45:20,922: INFO: common: created directory at: artifacts/data_ingestion]


TypeError: DataIngestionConfig.__init__() got an unexpected keyword argument 'local_data_file'